In [38]:
import pandas as pd
import os, sys

In [39]:
class RBP:
    def __init__(self,name,concentrations):
        self.name = name
        self.concentrations = concentrations

In [40]:
files = os.listdir('RBNS_training')+os.listdir('RBNS_testing')
FILES = [file for file in files if  file.startswith("RBP1_")]
FILES

['RBP1_1300nM.seq', 'RBP1_input.seq']

In [41]:
def generate_data(i,names):
    concentrations = list()
    for name in names: 
        if i > 16:
            addres =  'RBNS_testing/RBP{}_{}.seq'.format(i ,name) 
        else :
            addres =  'RBNS_training/RBP{}_{}.seq'.format(i ,name)
        print("reading " + addres)
        df = pd.read_csv(addres, sep="\t",usecols=[0],names=['sequence'])
        print("shape with N = " + df.shape)
        df = [seq for seq in df['sequence'] if all(ch != 'N' for ch in seq)] 
        df = pd.DataFrame(data=df,columns=['sequence'])
        print("shape without N = "+ df.shape)
        concentrations.append(df)
    return concentrations 

def label_data(RBP,labels):
    for j  in range(0,len(RBP)): 
        RBP[j]['label'] = labels[j]
    return RBP

def merge_data(RBP):
    data = RBP[0]
    for j  in range(1,len(RBP)): 
        data = data.append(RBP[j], ignore_index=True)
    return data



In [42]:
import numpy as np
# from string import maketrans    # in python free use str.maketrans isntead

def oneHot(string):
	trantab=str.maketrans('ACGTU','01233')
	string=str(string)
	data=[int(x) for x in list(string.translate(trantab))]
	return np.eye(4)[data]

In [44]:
i = 1 #RBPi 1<i<31
names = ['input','1300nM']
labels = [0,1]
RBP1 = generate_data(i,names)
RBP1 = label_data(RBP1,labels)
RBP1 = merge_data(RBP1)
RBP1 = RBP1.sample(frac=1)
RBP1

reading RBNS_training/RBP1_input.seq
(19572050, 1)
(19557759, 1)
reading RBNS_training/RBP1_1300nM.seq
(17443521, 1)
(17430737, 1)


,sequence,label
18667807,TGCAGTTGTCAACATTTCCA,0
18915317,CGTTATGTAGTTGAGGCGTA,0
35623937,AGAAGCAGGACACGTATAGT,1
36502215,CCGGCATGATGGAGGTAGAT,1
29900718,AATCCGAAACACTTAGCAGG,1
...,...,...
3556295,TCAAATCAAGGCTAGCTATT,0
3336260,GGTTAGAAAGGGAAAAACGT,0
13000270,AAGCTACTATGACTCTCTAT,0
27208947,AGATACACGCATACCACTGA,1


In [45]:
X = RBP1["sequence"]
Y = np.array(RBP1["label"])

In [55]:
X = np.array(list(map(oneHot,X)))
X.shape

(36988496, 20, 4)

In [48]:
y_result = pd.read_csv('RNCMPT_training/RBP{}.txt'.format(i), sep="\t", header=None)
y_result = np.asarray(y_result)
x_result= pd.read_csv('RNCMPT_training/seq.txt', sep="\t", header=None)

In [54]:
max_seq = len(max(x_result[0],key=len))
x_result =list(map(oneHot,x_result[0]))
padding_x_result = [np.pad(sequence, [(0,max_seq-len(sequence)),(0,0)], constant_values=0.25) for sequence in x_result]
padding_x_result = np.asarray(padding_x_result)
x_result = [seq[j:j+20] for j in range(0,22) for seq in padding_x_result]
x_result = np.asarray(x_result)

In [60]:
print(padding_x_result.shape)
print(x_result.shape)

(241357, 41, 4)
(5309854, 20, 4)


In [62]:
y_predict = np.split(y_predict,y_result.shape[0])
y_pred = [np.mean(result) for result in y_predict]

In [ ]:
from scipy.stats.stats import pearsonr 
print(pearsonr(y_result,y_pred.reshape(len(y_pred))))